## predict a single isolate strain/species

* requires inputs are from a representative set of the MTBC


In [8]:
import os, sys, io, random, subprocess, re
import string
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width',600)
import pylab as plt
import seaborn as sns
sns.set_context("notebook", rc={"font.size":12,"axes.titlesize":8,"axes.labelsize":8})

from IPython.display import Image
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

from mtbdiff import utils, analysis

## get ref assemblies

In [ ]:
reload(utils)
path='../training_genomes'
fetch_mtb_assemblies(data=filtered, path=path)

## run nucdiff on training genomes

In [ ]:
names = analysis.run_genomes(path, outpath='../training_results')
struct, snp = utils.get_nucdiff_results('../training_results', names)
struct['assembly'] = struct.label.apply(lambda x: x.split('.')[0],1)
struct = struct.merge(info[icols], left_on='assembly',right_on='Assembly_nover',how='left')

## prepare training matrix

In [ ]:
X = utils.sites_matrix(filtered, index=['descr'], columns=['Species','Strain'], freq=5)
X.columns = X.columns.get_level_values(0)
#X=X.reset_index(drop=True)
print(X[:5])
X=X.T
X=X.drop('orygis')

resp = X.index
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(resp)
print (list(le.classes_))
y = le.transform(resp)

## fit classifier

In [ ]:
def build_classifier(X, y, cols=[], kind='rf'):
    """generic method to build and test sklearn regressor using given dataset"""
    
    from sklearn.feature_selection import SelectPercentile
    sel = SelectPercentile(percentile=50)
    sel.fit(X,y)
    X_s = sel.transform(X)
    print (X_s.shape) 

    from sklearn.model_selection import train_test_split,cross_val_score,ShuffleSplit
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)#, stratify=y)
    print (len(X_train),len(X_test))
    if kind == 'rf':
        from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
        cl = RandomForestClassifier(criterion='gini', max_depth=4, max_features='auto', n_estimators=200)
        cl.fit(X_train, y_train)

        importances = cl.feature_importances_
        indices = np.argsort(importances)[::-1]
        names = X.columns
        print ('feature ranking:')
        for f in range(X.shape[1])[:10]:
            print("%d. %s (%f)" % (f + 1, names[indices[f]], importances[indices[f]]))        

    '''scores = cl.predict(X_test)
    from sklearn import metrics
    #print metrics.roc_auc_score(y_test, p)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, scores, pos_label=1)
    roc_auc = metrics.auc(fpr, tpr)    
    plt.figure(figsize=(6,6))
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.title('ROC neoepitope score')
    plt.legend()'''
    return cl

cl=build_classifier(X, y)